In [2]:
from src.data.components.feature_extractors import ProsodyFeatureExtractor

/opt/homebrew/Caskroom/miniforge/base/envs/prosody/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
LAB_ROOT = "/Users/lukas/Desktop/projects/MIT/data/LibriTTSCorpusLabel/debug"
WAV_ROOT = "/Users/lukas/Desktop/projects/MIT/data/LibriTTS/debug"
DATA_CACHE = "/Users/lukas/Desktop/projects/MIT/data/cache"

In [4]:
extractor = ProsodyFeatureExtractor(
    lab_root=LAB_ROOT,
    wav_root=WAV_ROOT,
    data_cache=DATA_CACHE,
    extract_f0=False,
    extract_energy=True,
    extract_duration=True,
    extract_word_duration=True,
    # word_duration_mode="absolute",
    extract_pause_after=True,
    extract_pause_before=True,
    extract_prominence=False,
)

Extracted features: ['f0', 'energy', 'word_duration', 'duration', 'pause_before', 'pause_after']


Readers:   0%|          | 0/1 [00:00<?, ?it/s]













Readers: 100%|██████████| 1/1 [00:06<00:00,  6.91s/it]


In [5]:
samples = extractor.samples

In [24]:
extractor.get_all_energy()

[[0.9744174978027097,
  1.5323009181521703,
  0.22986427823974626,
  0.1589510770788018,
  0.6525959191960851,
  0.519032471316287,
  0.1891885058919285,
  0.42210988354470014,
  -0.09423187093335993,
  0.1308158027781075,
  0.21327306490848713,
  -0.2755138839158893,
  -0.18475991621022556,
  -0.46863414496577266,
  -0.8445624410559862,
  -0.640878355012618,
  -0.6919442264214801],
 [0.39818335281692957,
  -0.02193718101532256,
  0.1382389500619575,
  0.3681953651887507,
  -0.15114079355422735],
 [0.9617553617599812,
  1.3598753307609615,
  0.3331202019531679,
  0.5303631295694408,
  0.2879944561629067,
  -0.04263824389703149,
  -0.5662913508545593,
  0.016252875845584304,
  -0.0002420890542546243,
  -0.44669717268668063],
 [0.3872701287636418,
  0.1254224821841051,
  1.0595818370208478,
  -0.15028133302527943,
  -0.8054567181342381,
  0.03418880531180331,
  0.6237825063438349,
  0.38057446104774356,
  0.012193556592144324,
  0.42537985798143524,
  0.4066032619314175,
  -0.60290893809

In [6]:
samples[0]

OrderedDict([('reader', '121'),
             ('book', '121726'),
             ('text',
              'Also, a popular contrivance whereby love-making may be suspended but not stopped during the picnic season.'),
             ('features',
              {'pause_before': [0.0,
                0.0,
                0.0,
                0.0,
                0.37,
                0.0,
                0.03,
                0.0,
                0.0,
                0.0,
                0.31,
                0.0,
                0.0,
                0.11,
                0.0,
                0.0,
                0.0],
               'pause_after': [0.0,
                0.0,
                0.0,
                0.37,
                0.0,
                0.03,
                0.0,
                0.0,
                0.0,
                0.31,
                0.0,
                0.0,
                0.11,
                0.0,
                0.0,
                0.0,
                0.0],
       

In [7]:
texts = extractor.get_all_text()
texts

['Also, a popular contrivance whereby love-making may be suspended but not stopped during the picnic season.',
 'The song of the wretched.',
 'Also, a draft on futurity, sometimes honored, but generally extended.',
 'Once held by Hobson and Dewey, now carried by Mother Eddy and Brother Dowie.',
 'A transferable ticket to the Haul of Fame.',
 'A Bill-Poster.',
 'The Scottish National Hymn.',
 "The beggar's plea, the politician's sceptre and the drummer's ablest assistant.",
 'Hence, full of strains.',
 'H',
 'Painful to hear.',
 'Tied to a woman.',
 'In Germany, they generally "Hock the Kaiser."']

In [8]:
energies = extractor.get_all_energy()

In [9]:
f0s = extractor.get_all_f0()

### Word Matching

In [10]:
# gpt tokenizer
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [11]:
sample = samples[3]

In [12]:
original = sample["text"]
original

'Once held by Hobson and Dewey, now carried by Mother Eddy and Brother Dowie.'

In [13]:
words = " ".join(sample["features"]["words"])
words

'once held by hobson and dewey now carried by mother eddy and brother dowie'

In [14]:
text_tokens = tokenizer.encode(original.lower(), return_tensors="pt")
text_tokens

tensor([[27078,  2714,   416, 32724,  1559,   290,   390,   732,    88,    11,
           783,  5281,   416,  2802,  1225,  9892,   290,  3956, 47276,   494,
            13]])

In [15]:
word_tokens = tokenizer.encode(words, return_tensors="pt")
word_tokens

tensor([[27078,  2714,   416, 32724,  1559,   290,   390,   732,    88,   783,
          5281,   416,  2802,  1225,  9892,   290,  3956, 47276,   494]])

In [16]:
import re


def split_string_by_punctuation(s: str):
    return re.findall(r"\b\w+\b|\S", s)

In [17]:
# gpt tokenizer
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [18]:
def assign_labels(input_string, labels):
    # Create list to hold words and punctuation
    words_with_punctuation = re.findall(r"[\w']+|[.,!?;\"-]|'", input_string)

    # Create list to hold only words
    words_only = re.findall(r"\w+'?\w*", input_string)

    # Make sure the number of labels matches the number of words
    if not len(labels) == len(words_only):
        # alignmend or extraction failed, skip sample
        return None, None, None

    # Create a generator for word-label pairs
    word_label_pairs = ((word, label) for word, label in zip(words_only, labels))

    # Create list of tuples where each word is matched to a label and each punctuation is matched to None
    words_with_labels = []
    for token in words_with_punctuation:
        if re.match(r"\w+'?\w*", token):
            words_with_labels.append(next(word_label_pairs))
        else:
            words_with_labels.append((token, None))

    return words_only, words_with_punctuation, words_with_labels

In [19]:
string = "However, this is well-typed."
labels = [0, 1, 2, 3, 4]

words, punct, labels = assign_labels(string, labels)
labels

[('However', 0),
 (',', None),
 ('this', 1),
 ('is', 2),
 ('well', 3),
 ('-', None),
 ('typed', 4),
 ('.', None)]

In [20]:
words, labels = zip(*labels)
words, labels

(('However', ',', 'this', 'is', 'well', '-', 'typed', '.'),
 (0, None, 1, 2, 3, None, 4, None))

### Helsinki prominence extractor test

In [21]:
from src.data.components.helsinki import HelsinkiProminenceExtractor

In [22]:
path = "/Users/lukas/Desktop/projects/MIT/MIT_prosody/data/Helsinki"
lab_path = "/Users/lukas/Desktop/projects/MIT/data/LibriTTSCorpusLabel/dev-clean"
file = "dev.txt"

In [23]:
extractor = HelsinkiProminenceExtractor(path, lab_path, file)

IsADirectoryError: [Errno 21] Is a directory: '/Users/lukas/Desktop/projects/MIT/data/LibriTTSCorpusLabel/dev-clean'

In [ ]:
texts = extractor.get_all_texts()

In [ ]:
prominences = extractor.get_all_real_prominence()

In [ ]:
texts[0]

"A 'JOLLY' ART CRITIC"

In [ ]:
extractor.index_to_filename

{0: '1272_128104_000001_000000',
 1: '1272_128104_000003_000000',
 2: '1272_128104_000003_000001',
 3: '1272_128104_000003_000002',
 4: '1272_128104_000003_000003',
 5: '1272_128104_000003_000005',
 6: '1272_128104_000005_000000',
 7: '1272_128104_000005_000001',
 8: '1272_128104_000005_000006',
 9: '1272_128104_000005_000007',
 10: '1272_128104_000005_000008',
 11: '1272_128104_000005_000009',
 12: '1272_128104_000005_000013',
 13: '1272_128104_000005_000014',
 14: '1272_128104_000006_000003',
 15: '1272_128104_000006_000008',
 16: '1272_128104_000006_000009',
 17: '1272_128104_000007_000000',
 18: '1272_128104_000007_000001',
 19: '1272_135031_000002_000000',
 20: '1272_135031_000005_000000',
 21: '1272_135031_000007_000001',
 22: '1272_135031_000008_000000',
 23: '1272_135031_000026_000002',
 24: '1272_135031_000028_000000',
 25: '1272_135031_000030_000000',
 26: '1272_135031_000032_000000',
 27: '1272_135031_000035_000000',
 28: '1272_135031_000035_000001',
 29: '1272_135031_000036

In [ ]:
extractor.get_filepath(0)

filename: 1272_128104_000001_000000
reader: 1272, book: 128104, ut: 000001_000000


### f0 test

In [7]:
from src.utils.text_processing import CelexReader

celex_path = "/Users/lukas/Desktop/projects/MIT/data/celex/celex.txt"

reader = CelexReader(celex_path)

In [8]:
reader.lookup("dog")

{'Class': 'V',
 'StressPattern': '1',
 'PhoneticSyllable': 'dQg',
 'MorphStatus': 'Z',
 'Frequency': '76'}

In [18]:
reader.get_stress_position("accelerate")

(0.25, 0.5)